In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import and modify dataset

In [ ]:
ds = pd.read_csv('../Dataset/data_abs.csv', header=None)
ds = ds[ds[4] == 1] # Select data for water media
ds = ds[ds[1] != 0] # Select only core-shell particles
ds = ds.sample(frac=1).reset_index(drop=True) # Randomize dataset

In [ ]:
num_mats = 11 # Total number of materials used (including "empty")
spectra = np.array(ds.iloc[:, 5:])/255 # Normalized spectra

m1 = np.array(ds[[0]]) # Core material ids
m2 = np.array(ds[[1]]) # Shell material ids
t1 = np.array(ds[[2]])/100 # Normalized core thickness
t2 = np.array(ds[[3]])/100 # Normalized shell thickness

geo = np.hstack((m1, m2, t1, t2)) # Geometry parameters

test_split = int(0.95*ds.shape[0]) # Number of samples separated for testing after training

# Training output
y_train = spectra[:test_split]
y_train = tf.expand_dims(y_train, axis=-1)

# Training features
x_train_m1 = keras.utils.to_categorical(m1[:test_split])
x_train_m1 = tf.expand_dims(x_train_m1, axis=1)
x_train_m2 = keras.utils.to_categorical(m2[:test_split])
x_train_m2 = tf.expand_dims(x_train_m2, axis=1)
x_train_t1 = t1[:test_split]
x_train_t1 = tf.expand_dims(x_train_t1, axis=-1)
x_train_t2 = t2[:test_split]
x_train_t2 = tf.expand_dims(x_train_t2, axis=-1)
x_train = geo[:test_split]
x_train = tf.expand_dims(x_train, axis=-1)

# Testing output
y_test = spectra[test_split:]
y_test = tf.expand_dims(y_test, axis=-1)

# Testing features
x_test_m1 = keras.utils.to_categorical(m1[test_split:])
x_test_m1 = tf.expand_dims(x_test_m1, axis=1)
x_test_m2 = keras.utils.to_categorical(m2[test_split:])
x_test_m2 = tf.expand_dims(x_test_m2, axis=1)
x_test_t1 = t1[test_split:]
x_test_t1 = tf.expand_dims(x_test_t1, axis=-1)
x_test_t2 = t2[test_split:]
x_test_t2 = tf.expand_dims(x_test_t2, axis=-1)
x_test = geo[test_split:]
x_test = tf.expand_dims(x_test, axis=-1)

# resNeXt block

A 1D resNeXt block that is subsequently used to define the full model. The concept modifies the classical res-block using grouped convolutions and bottleneck layers.

In [ ]:
def resNeXt_block(x_in, N_filter, N_groups, N_bottleneck, kernel_size=3, strides=1,
                   conv_layer=keras.layers.Conv1D, alpha=0.3, with_BN=True):

    # Residual connection
    if x_in.shape[-1] != N_filter or strides != 1:
        # If input != output dimension, add BN/ReLU/conv. into shortcut
        conv_shortcut = conv_layer(
            filters=N_filter, kernel_size=1, strides=strides, padding='same')(x_in)
    else:
        # Else use bare input as shortcut
        conv_shortcut = x_in
    
    if with_BN:
        conv_shortcut = keras.layers.BatchNormalization()(conv_shortcut)

    # ResNeXt path
    N_bottleneck_filters = int(N_filter * N_bottleneck)
    x = x_in

    x = conv_layer(filters=N_bottleneck_filters, kernel_size=1, strides=1,
                            padding='same', use_bias=not with_BN)(x)
    if with_BN:
        x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU(alpha)(x)

    x = conv_layer(filters=N_bottleneck_filters, kernel_size=kernel_size,
                   strides=strides, padding='same',
                   groups=N_bottleneck_filters//N_groups, use_bias=not with_BN)(x)
    if with_BN:
        x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU(alpha)(x)

    x = conv_layer(filters=N_filter, kernel_size=1, strides=1,
                   padding='same', use_bias=not with_BN)(x)
    if with_BN:
        x = keras.layers.BatchNormalization()(x)

    # Add residual and main and apply a further activation
    x = keras.layers.Add()([x, conv_shortcut])
    x = keras.layers.LeakyReLU(alpha)(x)

    return x

# Define the resNeXt model

In [ ]:
N_blocks = 2

# Input features
design_in_m1 = keras.Input(shape=x_train_m1.shape[1:], name='design_in_m1')
design_in_m2 = keras.Input(shape=x_train_m2.shape[1:], name='design_in_m2')
design_in_t1 = keras.Input(shape=x_train_t1.shape[1:], name='design_in_t1')
design_in_t2 = keras.Input(shape=x_train_t2.shape[1:], name='design_in_t2')

x_m1 = design_in_m1
x_m2 = design_in_m2
x_t1 = design_in_t1
x_t2 = design_in_t2

x_1 = keras.layers.Concatenate(axis=-1)([x_m1, x_t1])
x_2 = keras.layers.Concatenate(axis=-1)([x_m2, x_t2])
x = keras.layers.Concatenate(axis=1)([x_1, x_2])

x = tf.keras.layers.ZeroPadding1D((1,1))(x) # 2 --> 4

for i in range(N_blocks):
    x = resNeXt_block(x, N_filter=256, N_groups=8, N_bottleneck=4, strides=1)
x = keras.layers.UpSampling1D()(x) # 8 --> 16

for i in range(N_blocks):
    x = resNeXt_block(x, N_filter=128, N_groups=8, N_bottleneck=4, strides=1)
x = keras.layers.UpSampling1D(5)(x) # 8 --> 40

for i in range(N_blocks):
    x = resNeXt_block(x, N_filter=64, N_groups=8, N_bottleneck=4, strides=1)
x = keras.layers.UpSampling1D(5)(x) # 40 --> 200

for i in range(N_blocks):
    x = resNeXt_block(x, N_filter=32, N_groups=8, N_bottleneck=4, strides=1)

x = resNeXt_block(x, N_filter=32, N_groups=32,
                  N_bottleneck=4, strides=1, with_BN='False')

# Output
resnext_output = keras.layers.Conv1D(
    filters=1, kernel_size=1, padding='same')(x)

resnext_model = keras.models.Model(
    inputs=[design_in_m1,design_in_m2, design_in_t1, design_in_t2], outputs=resnext_output, name='resnext_model')

In [ ]:
def residual_block(x_in, N_filter, kernel_size=3, strides=1,
                   conv_layer=keras.layers.Conv1D, alpha=0.3,
                   with_BN=False):

    # Residual connection
    if x_in.shape[-1] != N_filter or strides != 1:
        # If input != output dimension, add BN/ReLU/conv. into shortcut
        conv_shortcut = conv_layer(
            filters=N_filter, kernel_size=1, strides=strides, padding='same')(x_in)
    else:
        # Else use bare input as shortcut
        conv_shortcut = x_in

    # Convolutional path
    x = x_in

    x = conv_layer(filters=N_filter, kernel_size=1, strides=1,
                   padding='same', use_bias=not with_BN)(x)
    if with_BN:
        x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU(alpha)(x)

    x = conv_layer(filters=N_filter, kernel_size=kernel_size,
                   strides=strides, padding='same', use_bias=not with_BN)(x)
    if with_BN:
        x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU(alpha)(x)

    x = conv_layer(filters=N_filter, kernel_size=1, strides=1,
                   padding='same', use_bias=not with_BN)(x)
    if with_BN:
        x = keras.layers.BatchNormalization()(x)

    # Add residual and main and apply a further activation
    x = keras.layers.Add()([x, conv_shortcut])
    x = keras.layers.LeakyReLU(alpha)(x)

    return x

# Train the resNeXt model

In [ ]:
# Compile with optimizer and cost function
resnext_model.compile(optimizer=keras.optimizers.AdamW(learning_rate=0.01),
                      loss='mse')

# Automatic learning rate reduction on loss plateau
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', factor=1/2, patience=4, verbose=1)

# Callback for early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True)

# Assemble training configurations
train_kwargs = dict(x=[x_train_m1, x_train_m2, x_train_t1, x_train_t2],
                    y=y_train, validation_split=0.2,
                    epochs=16, callbacks=[reduce_lr, early_stop])

# Fit the model with BN --> increasing batchsize schdedule
hist = None # Global history after BS incrase
for i in range(4): # 4x16 epochs, increasing batchsize
    _h = resnext_model.fit(batch_size=(16 * 2**i), **train_kwargs)
    if hist is None:
        hist = _h
    else:
        # Update history
        for k in hist.history:
            hist.history[k] = np.concatenate([hist.history[k], _h.history[k]])

# Plot losses

In [ ]:
plt.figure()
plt.plot(hist.history['loss'], label='loss')
plt.plot(hist.history['val_loss'], label='val_loss')
plt.yscale('log')
plt.legend()
plt.show()

# Save model

In [ ]:
resnext_model.save('../Models/model_tandem_forward.h5')